<a href="https://colab.research.google.com/github/Newborn7/MyFirstRepo/blob/main/%D1%80%D0%B0%D0%B7%D0%B4%D0%B5%D0%BB%204.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.3 MB/s eta 0:00:00


In [2]:
!pip install lightgbm -q

In [3]:
!pip install xgboost -q

In [4]:
import os
import random
import numpy as np

def set_all_seeds(seed):

    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

DEFAULT_RANDOM_SEED = 42
set_all_seeds(DEFAULT_RANDOM_SEED)

In [5]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from scipy import stats
from lightgbm import Dataset, LGBMRegressor, train
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
import xgboost as xgb


In [6]:
train = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv')
print("train", train.shape)
train.sample(3)

train (2337, 17)


,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
1720,z14411100u,Volkswagen ID.4,premium,electro,4.28,2017,114851,2020,59.18,engine_fuel,4.140287,1.163474e+07,0.1,190.915943,-4.022916,174,148
2025,n-6604555n,VW Polo VI,economy,petrol,5.00,2015,87354,2018,26.50,electro_bug,4.049598,1.797655e+07,0.0,200.000000,2.871492,174,171
1430,d-1051262p,Smart ForFour,economy,petrol,4.38,2013,37920,2015,27.18,engine_overheat,4.394770,1.543554e+07,0.1,183.260311,11.772687,174,171


In [ ]:
train.isna().sum()

car_id                      0
model                       0
car_type                    0
fuel_type                   0
car_rating                  0
year_to_start               0
riders                      0
year_to_work                0
target_reg                  0
target_class                0
mean_rating                 0
distance_sum                0
rating_min                  0
speed_max                   0
user_ride_quality_median    0
deviation_normal_count      0
user_uniq                   0
dtype: int64

In [70]:
test = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv')
print("test", test.shape)
test.sample(3)

test (1913, 16)


,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
391,V-4730687q,VW Tiguan,economy,petrol,4.10,2016,95799,2018,electro_bug,5.175172,1.361971e+07,0.36,110.108575,-5.733488,174,172
1818,Z16920973J,Hyundai Solaris,economy,petrol,4.42,2013,48112,2019,engine_check,4.450345,1.047056e+07,0.10,118.687177,10.073404,174,170
102,I-8201538D,Kia Rio X,economy,petrol,4.84,2013,38486,2019,engine_check,4.630402,1.190988e+07,0.14,103.661702,-15.105433,174,171


In [8]:
cat_features = ["model", "car_type", "fuel_type"]  # Выделяем категориальные признаки
target = ["target_reg"]
features2drop = ["car_id", "target_class"]  # эти фичи будут удалены

# Отбираем итоговый набор признаков для использования моделью
filtered_features = [i for i in train.columns if (i not in target and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

In [10]:
X = train[filtered_features]
y = train[target]

In [12]:
X

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,4.737759,1.214131e+07,0.10,180.855726,0.023174,174,170
1,VW Polo VI,economy,petrol,3.90,2015,78218,2021,4.480517,1.803909e+07,0.00,187.862734,12.306011,174,174
2,Renault Sandero,standart,petrol,6.30,2012,23340,2017,4.768391,1.588366e+07,0.10,102.382857,2.513319,174,173
3,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,3.880920,1.651883e+07,0.10,172.793237,-5.029476,174,170
4,Renault Sandero,standart,petrol,4.70,2012,26428,2017,4.181149,1.398317e+07,0.10,203.462289,-14.260456,174,171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,Smart ForFour,economy,petrol,4.38,2017,121239,2018,4.608908,1.739222e+07,0.10,141.502350,-6.624534,174,171
2333,Audi A4,premium,petrol,4.30,2016,107793,2020,4.683793,1.174052e+07,0.10,155.000000,-8.582467,174,169
2334,Kia Rio,economy,petrol,3.88,2015,80234,2019,4.655345,1.202022e+07,0.10,104.180940,-0.778524,174,172
2335,Renault Sandero,standart,petrol,4.50,2014,60048,2020,4.638333,1.788307e+07,0.10,200.000000,2.464975,174,171


**Обучаем CatBoostRegressor**

In [ ]:
CatBoostRegressor() #loss функция по умолчанию RMSE

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = CatBoostRegressor(thread_count=-1,
                         random_seed=42,
                         cat_features=cat_features)

clf.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    verbose=200,
    use_best_model=True,
    plot=False,
    early_stopping_rounds=100,
)

Learning rate set to 0.056174
0:	learn: 17.0065561	test: 17.7387430	best: 17.7387430 (0)	total: 50.1ms	remaining: 50.1s
200:	learn: 9.3900288	test: 12.1591227	best: 12.1374989 (148)	total: 736ms	remaining: 2.92s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 12.1374989
bestIteration = 148

Shrink model to first 149 iterations.


**Теперь обучаем на 3х фолдах**

In [40]:
n_splits = 3  # Число фолдов.
clfs = [] # Тут будем сохранять модели
scores = [] # Тут будем хранить скоры валидационных фолдов

# параметры валидации, обучение будет идти на n_splits фолдах
X = train[filtered_features]
y = train[target]

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Специальный класс для ускорения обучения
    train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
    eval_dataset = Pool(data=X_test, label=y_test, cat_features=cat_features)

    clf = CatBoostRegressor(
        #depth=4,
        iterations=3500,
        #learning_rate=0.06,
        loss_function="RMSE",

        # Главная фишка катбуста - работа с категориальными признаками
        cat_features=cat_features,
        # ignored_features = ignored_features,

        # Регуляризация и ускорение
        colsample_bylevel=0.098,
        subsample=0.95,
        l2_leaf_reg=9,
        min_data_in_leaf=243,
        max_bin=187,
        random_strength=1,

        # Параметры скорения
        task_type="CPU",
        thread_count=-1,
        bootstrap_type="Bernoulli",

        # Важное!
        random_seed=42,
        early_stopping_rounds=50)

    clfs.append(clf)

    clf.fit(
        train_dataset,
        eval_set=eval_dataset,
        verbose=500,
        use_best_model=True,
        plot=False)

    scores.append(np.mean([v for k, v in clf.best_score_["validation"].items()], dtype="float16"))
    # scores.append(clf.best_score_['validation']['MultiClass'])
    # clf.save_model("../tmp_data/cool_catboost_model_{}_deep".format(n))

assert len(clfs) == n_splits
print("mean  RMSE score --------->", np.mean(scores, dtype="float16") - np.std(scores, dtype="float16"))

0:	learn: 17.8581170	test: 17.2075124	best: 17.2075124 (0)	total: 47.7ms	remaining: 2m 46s
500:	learn: 10.7618273	test: 11.8655607	best: 11.8606813 (492)	total: 288ms	remaining: 1.72s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.85158627
bestIteration = 520

Shrink model to first 521 iterations.
0:	learn: 17.1967734	test: 18.5098462	best: 18.5098462 (0)	total: 277us	remaining: 971ms
500:	learn: 10.9608230	test: 12.3455125	best: 12.3455125 (500)	total: 238ms	remaining: 1.42s
1000:	learn: 9.9035013	test: 11.9706988	best: 11.9671948 (961)	total: 469ms	remaining: 1.17s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.95340053
bestIteration = 1098

Shrink model to first 1099 iterations.
0:	learn: 17.8655644	test: 17.1938065	best: 17.1938065 (0)	total: 283us	remaining: 992ms
500:	learn: 10.9821333	test: 11.7915646	best: 11.7915646 (500)	total: 241ms	remaining: 1.44s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.5529131
bestIt

In [41]:
clfs[2].save_model("cool_catboost_model1_deep") # сохраняем модель на лучшем фолде

In [51]:
catboost_preds = clfs[2].predict(test[filtered_features]) # делаем предсказания на модели на лучшем фолде
catboost_preds.shape

(1913,)

In [71]:
test['Catboost_target_reg'] = pd.DataFrame(catboost_preds)
test

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq,Catboost_target_reg
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,gear_stick,3.746207,1.407539e+07,0.10,195.454152,10.566220,174,170,46.913346
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016,engine_overheat,4.318966,1.970390e+07,0.00,181.538685,11.807941,174,174,31.960878
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017,electro_bug,5.134655,9.314946e+06,0.10,118.440645,14.862538,174,172,33.802347
3,F12725233R,Smart ForFour,economy,petrol,2.80,2014,68833,2021,engine_check,4.617356,9.336838e+06,0.83,112.829785,20.088904,174,172,35.979513
4,l-1139189J,Skoda Rapid,economy,petrol,6.56,2013,42442,2021,another_bug,4.287471,1.196250e+07,0.00,187.846088,3.698460,174,172,32.327644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,x13640960Q,Nissan Qashqai,standart,petrol,2.44,2015,81070,2016,engine_overheat,4.641379,8.035217e+06,0.10,161.000000,5.426149,174,171,57.644811
1909,Z-2276652N,Renault Kaptur,standart,petrol,4.44,2014,70067,2019,engine_overheat,3.956954,1.792881e+07,0.10,195.394955,-6.225923,174,172,45.007600
1910,F-2165841B,Hyundai Solaris,economy,petrol,5.16,2011,6574,2022,engine_check,4.607816,1.401163e+07,0.22,113.091551,-4.110241,174,172,33.821215
1911,x-1331529J,Hyundai Solaris,economy,petrol,5.76,2015,85648,2018,electro_bug,4.462644,1.375324e+07,0.10,154.105593,-4.337686,174,172,44.771408


In [49]:
test[['car_id', 'target_reg']].to_csv("catboost_kfold.csv", index = False)

**добавим еще предсказания с другого фолда**

In [ ]:
catboost_preds1 = clfs[0].predict(test[filtered_features]) # делаем предсказания на модели на лучшем фолде

In [ ]:
test['target_reg2'] = pd.DataFrame(catboost_preds1)
test

In [ ]:
test['target_reg_mean'] = test[['target_reg','target_reg2']].mean(axis=1)
test

In [ ]:
test[['car_id', 'target_reg_mean']].to_csv("catboost_kfold1.csv", index = False)

**переберем параметры CatBoosta с помощью randomized_search**

In [ ]:
model = CatBoostRegressor(
    random_seed=42,
    thread_count=-1,
    cat_features=cat_features,
    loss_function="RMSE",
    verbose=500,
    early_stopping_rounds=50
)

param_distribution = {
    "one_hot_max_size": stats.bernoulli(p=0.2, loc=2),
    "learning_rate": [0.03, 0.1, 0.3],
    "l2_leaf_reg": [2, 5, 7],
    "iterations": [500,1000,1500,2000,3000,3500,4000],
    "depth": stats.binom(n=10, p=0.2),
}
randomized_search_result = model.randomized_search(param_distribution, X_train, y_train)

In [ ]:
# Лучшие параметры модели из перебора
print(model.best_score_)
randomized_search_result["params"]

{'learn': {'RMSE': 9.235939109543445}}


{'l2_leaf_reg': 5,
 'iterations': 1500,
 'depth': 2.0,
 'one_hot_max_size': 2.0,
 'learning_rate': 0.1}

**Повторим обучение на 3х фолдах с подобранными параметрами**

In [ ]:
X = train[filtered_features]
X

In [ ]:
n_splits = 3  # Число фолдов.
clfs = [] # Тут будем сохранять модели
scores = [] # Тут будем хранить скоры валидационных фолдов

# параметры валидации, обучение будет идти на n_splits фолдах
X = train[filtered_features]
y = train[target]

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Специальный класс для ускорения обучения
    train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
    eval_dataset = Pool(data=X_test, label=y_test, cat_features=cat_features)

    clf = CatBoostRegressor(
        depth=2,
        iterations=1500,
        learning_rate=0.1,
        loss_function="RMSE",

        # Главная фишка катбуста - работа с категориальными признаками
        cat_features=cat_features,
        # ignored_features = ignored_features,

        # Регуляризация и ускорение
        #colsample_bylevel=0.098,
        #subsample=0.95,
        l2_leaf_reg=5,
        #min_data_in_leaf=243,
        #max_bin=187,
        random_strength=1,
        one_hot_max_size = 2.0,

        # Параметры скорения
        task_type="CPU",
        thread_count=-1,
        bootstrap_type="Bernoulli",

        # Важное!
        random_seed=42,
        early_stopping_rounds=50)

    clfs.append(clf)

    clf.fit(
        train_dataset,
        eval_set=eval_dataset,
        verbose=500,
        use_best_model=True,
        plot=False)

    scores.append(np.mean([v for k, v in clf.best_score_["validation"].items()], dtype="float16"))
    # scores.append(clf.best_score_['validation']['MultiClass'])
    # clf.save_model("../tmp_data/cool_catboost_model_{}_deep".format(n))

assert len(clfs) == n_splits
print("mean  RMSE score --------->", np.mean(scores, dtype="float16") - np.std(scores, dtype="float16"))

0:	learn: 17.2155998	test: 16.6435452	best: 16.6435452 (0)	total: 775us	remaining: 1.16s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.92320256
bestIteration = 84

Shrink model to first 85 iterations.
0:	learn: 16.7371785	test: 18.0594812	best: 18.0594812 (0)	total: 693us	remaining: 1.04s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.8138425
bestIteration = 381

Shrink model to first 382 iterations.
0:	learn: 17.2215421	test: 16.6136503	best: 16.6136503 (0)	total: 1.88ms	remaining: 2.81s
500:	learn: 10.5252539	test: 11.4421385	best: 11.4270973 (450)	total: 327ms	remaining: 652ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.42709727
bestIteration = 450

Shrink model to first 451 iterations.
mean  RMSE score ---------> 11.51


**переберем параметры CatBoosta с помощью grid_search**

In [ ]:
model = CatBoostRegressor(
    random_seed=42,
    thread_count=-1,
    cat_features=cat_features,
    loss_function="RMSE",
    verbose=500,
    early_stopping_rounds=50
)

grid = {
    #"one_hot_max_size": stats.bernoulli(p=0.2, loc=2),
    "learning_rate": [0.03, 0.1, 0.3],
    "l2_leaf_reg": [2, 5, 7],
    "iterations": [500,1000,1500,2000,3000,3500,4000],
    "depth": [2,3,4,5,6,7],
}
grid_search_result = model.grid_search(grid, X_train, y_train)

In [ ]:
grid_search_result['params']

{'depth': 4, 'l2_leaf_reg': 5, 'iterations': 500, 'learning_rate': 0.1}

In [ ]:
grid_search_result['cv_results']

In [ ]:
n_splits = 3  # Число фолдов.
clfs = [] # Тут будем сохранять модели
scores = [] # Тут будем хранить скоры валидационных фолдов

# параметры валидации, обучение будет идти на n_splits фолдах
X = train[filtered_features]
y = train[target]

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Специальный класс для ускорения обучения
    train_dataset = Pool(data=X_train, label=y_train, cat_features=cat_features)
    eval_dataset = Pool(data=X_test, label=y_test, cat_features=cat_features)

    clf = CatBoostRegressor(
        depth=4,
        iterations=500,
        learning_rate=0.1,
        loss_function="RMSE",

        # Главная фишка катбуста - работа с категориальными признаками
        cat_features=cat_features,
        # ignored_features = ignored_features,

        # Регуляризация и ускорение
        #colsample_bylevel=0.098,
        #subsample=0.95,
        l2_leaf_reg=5,
        #min_data_in_leaf=243,
        #max_bin=187,
        random_strength=1,
        one_hot_max_size = 2.0,

        # Параметры скорения
        task_type="CPU",
        thread_count=-1,
        bootstrap_type="Bernoulli",

        # Важное!
        random_seed=42,
        early_stopping_rounds=50)

    clfs.append(clf)

    clf.fit(
        train_dataset,
        eval_set=eval_dataset,
        verbose=500,
        use_best_model=True,
        plot=False)

    scores.append(np.mean([v for k, v in clf.best_score_["validation"].items()], dtype="float16"))
    # scores.append(clf.best_score_['validation']['MultiClass'])
    # clf.save_model("../tmp_data/cool_catboost_model_{}_deep".format(n))

assert len(clfs) == n_splits
print("mean  RMSE score --------->", np.mean(scores, dtype="float16") - np.std(scores, dtype="float16"))

0:	learn: 17.0531506	test: 16.4590357	best: 16.4590357 (0)	total: 1.65ms	remaining: 823ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.88635113
bestIteration = 60

Shrink model to first 61 iterations.
0:	learn: 16.5273098	test: 17.7806562	best: 17.7806562 (0)	total: 1.49ms	remaining: 743ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.88512237
bestIteration = 92

Shrink model to first 93 iterations.
0:	learn: 17.0587507	test: 16.4515632	best: 16.4515632 (0)	total: 1.51ms	remaining: 754ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 11.41194785
bestIteration = 201

Shrink model to first 202 iterations.
mean  RMSE score ---------> 11.51


**обучаем LightGBMRegressor (goss)**

In [53]:
# переведем категориальные фичи в int с помощью  LabelEncoder

le = LabelEncoder()
for col in cat_features:
    X[col] = le.fit_transform(X[col])

<ipython-input-53-ee1fbe18564d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-53-ee1fbe18564d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-53-ee1fbe18564d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,)

In [55]:
train_data = Dataset(
    X_train,
    y_train,
    categorical_feature=cat_features,
    free_raw_data=False,
)

# Еще валидационный датасет для настройки ранней остановки
val_data = Dataset(
    X_test,
    y_test,
    categorical_feature=cat_features,
    free_raw_data=False,
)

In [ ]:
gbm_params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    'boosting_type': "goss",
    "num_iterations": 10000
}

In [ ]:
# num_round = 1000
# cv_results = lgb.cv(
#     params,
#     train_data,
#     num_boost_round=num_round,
#     nfold=3,
#     early_stopping_rounds=100,
#     stratified=False,
# )

# best_round = len(cv_results["rmse-mean"])
# model = lgb.train(params, train_data, num_boost_round=best_round)

In [ ]:
gbm = lgb.LGBMRegressor(**gbm_params)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=100)

In [ ]:
gbm.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('rmse', 12.178725017789267)])})

In [ ]:
gbm.best_iteration_

85

**LightGBM_gridsearch**

In [ ]:
model = lgb.LGBMRegressor()

param_dist = {"max_depth": [4,5,6,10],
              "learning_rate" : [0.05, 0.1,0.15],
              "num_leaves": [15, 20, 25, 32, 40],
              "n_estimators": [40, 60, 80, 100, 200],
              "random_state": [42]}

grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3,
                                   verbose=10, n_jobs=-1)


grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


GridSearchCV(cv=3, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [4, 5, 6, 10],
                         'n_estimators': [40, 60, 80, 100, 200],
                         'num_leaves': [15, 20, 25, 32, 40],
                         'random_state': [42]},
             verbose=10)

In [ ]:
max_depth_best = grid_search.best_estimator_.max_depth
num_leaves_best = grid_search.best_estimator_.num_leaves
n_estimators_best = grid_search.best_estimator_.n_estimators
learning_rate_best = grid_search.best_estimator_.learning_rate

print(max_depth_best, num_leaves_best, n_estimators_best, learning_rate_best)

4 15 80 0.1


In [ ]:
gbm_params_optimised = {
    "objective": "regression",
    "metric": "rmse",
    "max_depth": max_depth_best,
    "num_leaves": num_leaves_best,
    "learning_rate": learning_rate_best,
    "feature_fraction": 0.9,
    "n_estimators": n_estimators_best,
    'boosting_type': "goss",
    "num_iterations": 10000
}

In [ ]:
gbm = lgb.LGBMRegressor(**gbm_params_optimised)

In [ ]:
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=100)

In [ ]:
gbm.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('rmse', 12.028295389193095)])})

**Повторим обучение Lightgbm на 3х фолдах с подобранными параметрами**

In [ ]:
train

In [ ]:
n_splits = 3  # Число фолдов.
gbms = [] # Тут будем сохранять модели
gbm_scores = [] # Тут будем хранить скоры валидационных фолдов

# параметры валидации, обучение будет идти на n_splits фолдах
X = train[filtered_features]
y = train[target]

le = LabelEncoder()
for col in cat_features:
    X[col] = le.fit_transform(X[col])

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Специальный класс для ускорения обучения
    train_data = Dataset(
    X_train,
    y_train,
    categorical_feature=cat_features,
    free_raw_data=False)


    val_data = Dataset(
    X_test,
    y_test,
    categorical_feature=cat_features,
    free_raw_data=False)

    gbm_params_optimised = {
    "objective": "regression",
    "metric": "rmse",
    "max_depth": 4,
    "num_leaves": 15,
    "learning_rate": 0.1,
    "feature_fraction": 0.9,
    "n_estimators": 80,
    'boosting_type': "goss",
    "num_iterations": 10000,
    "random_state": [42]}

    gbm = lgb.LGBMRegressor(**gbm_params_optimised)

    gbms.append(gbm)

    gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        early_stopping_rounds=100)

    #gbm_scores.append(np.mean([v for k, v in gbm.best_score_["validation"].items()], dtype="float16"))
    gbm_scores.append(gbm.best_score_['valid_0'].values())


assert len(gbms) == n_splits
#print("mean  RMSE score --------->", np.mean(gbm_scores, dtype="float16") - np.std(gbm_scores, dtype="float16"))
print("mean  RMSE score --------->", np.mean([list(values) for values in gbm_scores]))

In [57]:
gbm.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('rmse', 11.852979722088289)])})

In [58]:
for values in gbm_scores:
  print(list(values))

[12.074375673494051]
[12.163769833561092]
[11.852979722088289]


In [ ]:
mean_value = np.mean([list(values) for values in gbm_scores])
mean_value

12.030375076381143

In [61]:
gbms[2].booster_.save_model('cool_lightgbm_model') # сохраняем модель на лучшем фолде

In [72]:
# трансформируем тестовый датасет, также как и train

X_test = test[filtered_features]

le = LabelEncoder()
for col in cat_features:
    X_test[col] = le.fit_transform(X_test[col])

<ipython-input-72-6984d434f1ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = le.fit_transform(X_test[col])
<ipython-input-72-6984d434f1ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = le.fit_transform(X_test[col])
<ipython-input-72-6984d434f1ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [73]:
loaded_booster = lgb.Booster(model_file="cool_lightgbm_model")

In [74]:
lightgbm_preds = loaded_booster.predict(X_test) # делаем предсказания на модели на лучшем фолде
lightgbm_preds.shape

(1913,)

In [75]:
test['lgb_target_reg'] = pd.DataFrame(lightgbm_preds)
test

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq,Catboost_target_reg,lgb_target_reg
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,gear_stick,3.746207,1.407539e+07,0.10,195.454152,10.566220,174,170,46.913346,49.735647
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016,engine_overheat,4.318966,1.970390e+07,0.00,181.538685,11.807941,174,174,31.960878,35.734030
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017,electro_bug,5.134655,9.314946e+06,0.10,118.440645,14.862538,174,172,33.802347,31.434782
3,F12725233R,Smart ForFour,economy,petrol,2.80,2014,68833,2021,engine_check,4.617356,9.336838e+06,0.83,112.829785,20.088904,174,172,35.979513,34.328350
4,l-1139189J,Skoda Rapid,economy,petrol,6.56,2013,42442,2021,another_bug,4.287471,1.196250e+07,0.00,187.846088,3.698460,174,172,32.327644,31.396779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,x13640960Q,Nissan Qashqai,standart,petrol,2.44,2015,81070,2016,engine_overheat,4.641379,8.035217e+06,0.10,161.000000,5.426149,174,171,57.644811,59.154587
1909,Z-2276652N,Renault Kaptur,standart,petrol,4.44,2014,70067,2019,engine_overheat,3.956954,1.792881e+07,0.10,195.394955,-6.225923,174,172,45.007600,52.151615
1910,F-2165841B,Hyundai Solaris,economy,petrol,5.16,2011,6574,2022,engine_check,4.607816,1.401163e+07,0.22,113.091551,-4.110241,174,172,33.821215,38.186486
1911,x-1331529J,Hyundai Solaris,economy,petrol,5.76,2015,85648,2018,electro_bug,4.462644,1.375324e+07,0.10,154.105593,-4.337686,174,172,44.771408,45.587600


In [ ]:
test[['car_id', 'target_reg']].to_csv("lightgbm_kfold.csv", index = False)

**XGBoostRegressor (dart)**

In [15]:
X.shape

(2337, 14)

In [17]:
print(X_train.shape, X_test.shape)

(1869, 14) (468, 14)


In [16]:
# Кат фичи преведем в тип 'category'
for col in cat_features:
    X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-16-d6fba6c7c7bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
<ipython-input-16-d6fba6c7c7bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
<ipython-input-16-d6fba6c7c7bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [ ]:
dtrain = xgb.DMatrix(X_train,
                     label=y_train,
                     nthread=-1,
                     enable_categorical=True,)

dtest = xgb.DMatrix(X_test,
                    y_test,
                    nthread=-1,
                    enable_categorical=True)

In [ ]:
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist', # 'gpu_hist' # Место для ускорения
    'seed': 42,
}
booster = xgb.train(params,
                    dtrain=dtrain,
                    num_boost_round=30,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
                    early_stopping_rounds=10,
                    verbose_eval=5)

[0]	dtrain-rmse:34.42113	dtest-rmse:35.60853
[5]	dtrain-rmse:11.37870	dtest-rmse:13.99612
[10]	dtrain-rmse:8.48258	dtest-rmse:12.48961
[15]	dtrain-rmse:7.36409	dtest-rmse:12.51914
[20]	dtrain-rmse:6.74858	dtest-rmse:12.56913
[24]	dtrain-rmse:6.16001	dtest-rmse:12.71189


In [ ]:
reg = xgb.XGBRegressor(booster = 'dart',
                       tree_method="hist",
                       enable_categorical=True,
                       use_rmm = True,
                       rate_drop = 0.25,
                       #skip_drop = 0.5
                       )

# fit() не принимает DMatrix
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster='dart', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
y_pred = reg.predict(X_test)
print(mean_squared_error(y_test, y_pred) ** 0.5)

12.15666099937917


In [ ]:
# Создадим список learning rates с помощью функции np.linspace
iters = 30
learning_rates = np.linspace(0.3, 0.005, iters).tolist() # значение lr будет плавно понижаться каждую итерацию с 0.3 до 0.005
scheduler = xgb.callback.LearningRateScheduler(learning_rates)

params = {
    'booster': 'dart',
    'rate_drop': 0.1,
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'seed': 42,
}
booster = xgb.train(params,
                    dtrain=dtrain,
                    num_boost_round=iters,
                    evals=[(dtrain, 'dtrain'), (dtest, 'dtest')],
                    callbacks=[scheduler], # <------ ✅
                    early_stopping_rounds=10,
                    verbose_eval=5)

y_pred = booster.predict(dtest, iteration_range=(0, booster.best_iteration + 1))
mean_squared_error(y_test, y_pred) ** 0.5

[0]	dtrain-rmse:34.42113	dtest-rmse:35.60853
[5]	dtrain-rmse:12.70756	dtest-rmse:15.16798
[10]	dtrain-rmse:9.79383	dtest-rmse:12.84091
[15]	dtrain-rmse:9.20776	dtest-rmse:12.45966
[20]	dtrain-rmse:8.88157	dtest-rmse:12.34695
[25]	dtrain-rmse:8.69940	dtest-rmse:12.28216
[29]	dtrain-rmse:8.67369	dtest-rmse:12.27622


12.276214995480153

**Grid Search with XGBoost**

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'seed': [42],
        }

In [ ]:
reg = xgb.XGBRegressor(booster = 'dart',
                       tree_method="hist",
                       enable_categorical=True,
                       use_rmm = True,
                       rate_drop = 0.25,
                       #skip_drop = 0.5
                       )

In [ ]:
grid_search = GridSearchCV(reg, param_grid=params, cv = 3,
                                   verbose=10, n_jobs=-1)


grid_search.fit(X_train, y_train)   # считается больше часа

Fitting 3 folds for each of 405 candidates, totalling 1215 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster='dart',
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None,...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5],
                         'min_child_weight': [1, 5, 10], 'seed': [42],
                         'subsample': [0.6, 0.8, 1.0]},
             verbose=10)

In [ ]:
min_child_weight = grid_search.best_estimator_.min_child_weight
gamma = grid_search.best_estimator_.gamma
subsample = grid_search.best_estimator_.subsample
colsample_bytree = grid_search.best_estimator_.colsample_bytree
max_depth = grid_search.best_estimator_.max_depth

print(min_child_weight, gamma, subsample, colsample_bytree, max_depth)

10 1.5 1.0 1.0 5


In [ ]:
grid_search.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1.5,
 'max_depth': 5,
 'min_child_weight': 10,
 'seed': 42,
 'subsample': 1.0}

**Обучаем XGBoost на трех фолдах с подобранными параметрами**

In [ ]:
n_splits = 3  # Число фолдов.
xgbs = [] # Тут будем сохранять модели
xgb_scores = [] # Тут будем хранить скоры валидационных фолдов

for col in cat_features:
    X[col] = X[col].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    xgb_optimized_params = {
        'min_child_weight': 10,
        'gamma': 1.5,
        'subsample': 1.0,
        'colsample_bytree': 1.0,
        'max_depth': 5,
        'seed': 42,
        }

    reg = xgb.XGBRegressor(**xgb_optimized_params,
                           booster = 'dart',
                           tree_method="hist",
                           enable_categorical=True,
                           use_rmm = True,
                           rate_drop = 0.25,)




    xgbs.append(reg)

    reg.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            eval_metric='rmse',
            early_stopping_rounds=100)

    xgb_scores.append(reg.best_score)
    # scores.append(clf.best_score_['validation']['MultiClass'])
    # clf.save_model("../tmp_data/cool_catboost_model_{}_deep".format(n))

assert len(xgbs) == n_splits
print("mean  RMSE score --------->", np.mean(xgb_scores, dtype="float16"))

In [19]:
xgb_scores

[12.180290173651958, 12.118617690760546, 11.501494591384972]

In [21]:
xgbs[2].save_model("xgb_regressor.json")

In [ ]:
X_test = test[filtered_features]
for col in cat_features:
    X_test[col] = X_test[col].astype('category')

In [77]:
xgbs_pred = xgbs[2].predict(X_test)
xgbs_pred.shape

(1913,)

In [78]:
test['xgb_target_reg'] = pd.DataFrame(xgbs_pred)
test

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq,Catboost_target_reg,lgb_target_reg,xgb_target_reg
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,gear_stick,3.746207,1.407539e+07,0.10,195.454152,10.566220,174,170,46.913346,49.735647,42.683758
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016,engine_overheat,4.318966,1.970390e+07,0.00,181.538685,11.807941,174,174,31.960878,35.734030,33.588177
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017,electro_bug,5.134655,9.314946e+06,0.10,118.440645,14.862538,174,172,33.802347,31.434782,32.672295
3,F12725233R,Smart ForFour,economy,petrol,2.80,2014,68833,2021,engine_check,4.617356,9.336838e+06,0.83,112.829785,20.088904,174,172,35.979513,34.328350,31.986582
4,l-1139189J,Skoda Rapid,economy,petrol,6.56,2013,42442,2021,another_bug,4.287471,1.196250e+07,0.00,187.846088,3.698460,174,172,32.327644,31.396779,31.959963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,x13640960Q,Nissan Qashqai,standart,petrol,2.44,2015,81070,2016,engine_overheat,4.641379,8.035217e+06,0.10,161.000000,5.426149,174,171,57.644811,59.154587,54.173714
1909,Z-2276652N,Renault Kaptur,standart,petrol,4.44,2014,70067,2019,engine_overheat,3.956954,1.792881e+07,0.10,195.394955,-6.225923,174,172,45.007600,52.151615,44.656506
1910,F-2165841B,Hyundai Solaris,economy,petrol,5.16,2011,6574,2022,engine_check,4.607816,1.401163e+07,0.22,113.091551,-4.110241,174,172,33.821215,38.186486,33.653713
1911,x-1331529J,Hyundai Solaris,economy,petrol,5.76,2015,85648,2018,electro_bug,4.462644,1.375324e+07,0.10,154.105593,-4.337686,174,172,44.771408,45.587600,48.113819


In [80]:
test['target_reg'] = test[['Catboost_target_reg','lgb_target_reg', 'xgb_target_reg']].mean(axis=1) #Делаем итоговое предсказание на тестовом датасете, усреднив предсказание от 3-х лучших моделей.
test

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq,Catboost_target_reg,lgb_target_reg,xgb_target_reg,target_reg
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,gear_stick,3.746207,1.407539e+07,0.10,195.454152,10.566220,174,170,46.913346,49.735647,42.683758,46.444250
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016,engine_overheat,4.318966,1.970390e+07,0.00,181.538685,11.807941,174,174,31.960878,35.734030,33.588177,33.761028
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017,electro_bug,5.134655,9.314946e+06,0.10,118.440645,14.862538,174,172,33.802347,31.434782,32.672295,32.636474
3,F12725233R,Smart ForFour,economy,petrol,2.80,2014,68833,2021,engine_check,4.617356,9.336838e+06,0.83,112.829785,20.088904,174,172,35.979513,34.328350,31.986582,34.098148
4,l-1139189J,Skoda Rapid,economy,petrol,6.56,2013,42442,2021,another_bug,4.287471,1.196250e+07,0.00,187.846088,3.698460,174,172,32.327644,31.396779,31.959963,31.894795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,x13640960Q,Nissan Qashqai,standart,petrol,2.44,2015,81070,2016,engine_overheat,4.641379,8.035217e+06,0.10,161.000000,5.426149,174,171,57.644811,59.154587,54.173714,56.991037
1909,Z-2276652N,Renault Kaptur,standart,petrol,4.44,2014,70067,2019,engine_overheat,3.956954,1.792881e+07,0.10,195.394955,-6.225923,174,172,45.007600,52.151615,44.656506,47.271907
1910,F-2165841B,Hyundai Solaris,economy,petrol,5.16,2011,6574,2022,engine_check,4.607816,1.401163e+07,0.22,113.091551,-4.110241,174,172,33.821215,38.186486,33.653713,35.220471
1911,x-1331529J,Hyundai Solaris,economy,petrol,5.76,2015,85648,2018,electro_bug,4.462644,1.375324e+07,0.10,154.105593,-4.337686,174,172,44.771408,45.587600,48.113819,46.157609


In [81]:
test[['car_id', 'target_reg']].to_csv("3boostings_kfold.csv", index = False)

Выводы:


*   получены предсказания с помощью трех моделей градиентного бустинга: Catboost, LightGBM. XGBoost;
*  c помощью grid_search на всех трех моделях осущетсвлен подбор параметров;
*   С подобранными параметрами осущетсвлено обучение моделей на трех фолдах;
*   Итоговое предсказание получено на тестовом датасете, усреднив предсказания трех наилучших моделей.



